In [1]:
import piheaan
import os
import numpy as np

In [2]:
params = piheaan.ParameterPreset.FGb 
context = piheaan.make_context(params) 
piheaan.make_bootstrappable(context) 

In [3]:
#튜토리얼 파일에서 생성된 키 받아오기
sk = piheaan.SecretKey(context) 
key_file_path = "./key" 
log_num_slot = piheaan.get_log_full_slots(context) 
num_slot = 1 << log_num_slot 
sk = piheaan.SecretKey(context,key_file_path+"/secretkey.bin")

keypack = piheaan.KeyPack(context, key_file_path+"/") 
keypack.load_enc_key() 
keypack.load_mult_key() 
eval = piheaan.HomEvaluator(context,keypack) 
dec = piheaan.Decryptor(context) 
enc = piheaan.Encryptor(context)

In [4]:
#0~num_slot*0.0001까지 1씩 증가하게 암호문 만들기
input_msg = piheaan.Message(log_num_slot)
for k in range(num_slot):
    input_msg[k] = k*0.0001

input_ctxt = piheaan.Ciphertext(context)
enc.encrypt(input_msg, keypack, input_ctxt)

In [5]:
# 평균 함수
# 1. 전체 슬롯의 합 구하기
def enc_mean(input_ctxt):
    duplicate_input = piheaan.Ciphertext(input_ctxt)

    tmp_ctxt = piheaan.Ciphertext(context)
    for i in range(int(np.log2(num_slot))):
        eval.left_rotate(duplicate_input, 2**i, tmp_ctxt)
        eval.bootstrap(duplicate_input,duplicate_input)
        eval.bootstrap(tmp_ctxt,tmp_ctxt)

        eval.add(duplicate_input, tmp_ctxt, duplicate_input)

    # 잘 더해졌는지 확인
    # check_msg = piheaan.Message(log_num_slot)
    # dec.decrypt(input_ctxt,sk,check_msg)
    # print(check_msg[:5])

    # 2. 전체 개수로 나눠주기
    eval.mult(duplicate_input,1/num_slot,duplicate_input)
    eval.bootstrap(duplicate_input,duplicate_input)

    return duplicate_input

mean_ctxt = enc_mean(input_ctxt)
mean_msg = piheaan.Message(log_num_slot)
dec.decrypt(mean_ctxt,sk,mean_msg)

# 평문에서의 평균 구하기
msg = [i*0.0001 for i in range(0,num_slot)]
plain_mean = np.mean(msg)

print("동형암호 연산 평균 결과 : ", mean_msg[0].real)
print("평문 연산 평균 결과 : ", plain_mean)

동형암호 연산 평균 결과 :  1.6383500000000002
평문 연산 평균 결과 :  1.63835


In [6]:
# 분산 구하기 (제평-평제)
# 1. 인풋 제곱의 평균 구하기
square_ctxt = piheaan.Ciphertext(input_ctxt)
eval.mult(input_ctxt,square_ctxt,square_ctxt)

eval.bootstrap(square_ctxt,square_ctxt)

square_mean = enc_mean(square_ctxt)

# 2. 평균의 제곱 구하기
mean_square = piheaan.Ciphertext(context)
eval.mult(mean_ctxt,mean_ctxt,mean_square)

eval.bootstrap(mean_square,mean_square)

# 3. 1-2
var_ctxt = piheaan.Ciphertext(context)
eval.mult(mean_square,-1,mean_square)

eval.add(square_mean,mean_square,var_ctxt)
var_msg = piheaan.Message(log_num_slot)
dec.decrypt(var_ctxt,sk,var_msg)

# 평문에서의 분산 구하기
plain_var = np.var(msg,ddof=1)

print("동형암호 연산 분산 결과 : ", var_msg[0].real)
print("평문 연산 분산 결과 : ", plain_var)

동형암호 연산 분산 결과 :  0.8947848524999991
평문 연산 분산 결과 :  0.8948121600000002


In [7]:
# 표준편차

std_ctxt = piheaan.Ciphertext(context)
piheaan.math.approx.sqrt(eval,var_ctxt,std_ctxt)

std_msg = piheaan.Message(log_num_slot)
dec.decrypt(std_ctxt,sk,std_msg)

# 평문에서의 표준편차 구하기
plain_std = np.std(msg,ddof=1)

print("동형암호 연산 표준편차 결과 : ", std_msg[0].real)
print("평문 연산 표준편차 결과 : ", plain_std)

동형암호 연산 표준편차 결과 :  0.9459306805997991
평문 연산 표준편차 결과 :  0.9459451146868936
